In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import json
import os
import operator

In [2]:
exchange_folder = "../StackExchange/"
st_folder = "../StackExchange/Stackoverflow/"

## Extraction from original XML dump and convert into multiple json files

In [ ]:
dict_list = []
counter = 0
data_total_limit = 60000000
data_per_tag_limit = 1000
tag_dict = {}
path = "../../../../Stackoverflow_dump/Stackoverflow.com-Posts/Posts.xml"

In [8]:
for _, elem in ET.iterparse(path, events=("end",)):
    if elem.tag == "row" and counter < data_total_limit:
        if 'Tags' in elem.attrib:
            tag_string = elem.attrib["Tags"].replace("<","")
            tag_string = tag_string.replace(">"," ")
            overlimit = True
            for tag in tag_string.strip().split(" "):
                tag_dict[tag] = tag_dict.setdefault(tag, 0) + 1
                if tag_dict[tag] < data_per_tag_limit:
                    overlimit = False
            
            if overlimit:
                elem.clear()
                continue
                
        dict_list.append(elem.attrib)
        
    elem.clear()
    counter += 1
    
    if len(dict_list) % 50000 == 0:
        with open("../Stackoverflow/DataSelection/st_data_"+str(counter)+".json", "w") as outfile:
            json.dump(dict_list, outfile)
        dict_list = []
        
#df = pd.DataFrame(dict_list)

In [9]:
with open("../Stackoverflow/DataSelection/st_data_"+str(counter)+".json", "w") as outfile:
        json.dump(dict_list, outfile)

In [10]:
len(tag_dict.keys())

64101

In [42]:
with open(st_folder+"st_tags_tf.json", "w") as outfile:
    json.dump(tag_dict, outfile)

### Other sites

In [15]:
def XML_extraction(data_per_tag_limit, path, name, tag_diff):
    tag_dict = {}
    dict_list = []
    
    for _, elem in ET.iterparse(path+"Posts.xml", events=("end",)):
        if elem.tag == "row":
            if 'Tags' in elem.attrib:
                tag_string = elem.attrib["Tags"].replace("<","")
                tag_string = tag_string.replace(">"," ")
                overlimit = True
                for tag in tag_string.strip().split(" "):
                    if tag in tag_diff:
                        tag_dict[tag] = tag_dict.setdefault(tag, 0) + 1
                    
                        if tag_dict[tag] < data_per_tag_limit:
                            overlimit = False
            
                if overlimit:
                    elem.clear()
                    continue
                
            dict_list.append(elem.attrib)
        
        elem.clear()
    
    with open("../StackExchange/other_sites/"+name+".json", "w") as outfile:
        json.dump(dict_list, outfile)
        
    return tag_dict

In [16]:
data_per_tag_limit = 100
path = "../../../../Stackoverflow_dump/apple.stackexchange.com/"

In [17]:
df_st = pd.read_csv("../StackExchange/site_tags/stackoveflow_tag.csv")
df_ad = pd.read_csv("../StackExchange/site_tags/askdifferent_tag.csv")

tag_diff = set(df_ad["TagName"]).difference(set(df_st["TagName"]))

In [18]:
print(len(list(tag_diff)))

377


In [19]:
apple_dict = XML_extraction(data_per_tag_limit, path, "askdifferent", list(tag_diff))

In [20]:
len(apple_dict.keys())

377

In [22]:
with open("../StackExchange/other_sites/ad_tags_tf.json", "w") as outfile:
    json.dump(apple_dict, outfile)

In [43]:
data_per_tag_limit = 100
path = "../../../../Stackoverflow_dump/askubuntu.com/"

df_st = pd.read_csv("../StackExchange/site_tags/stackoveflow_tag.csv")
df_au = pd.read_csv("../StackExchange/site_tags/askubuntu_tag.csv")

tag_diff = set(df_au["TagName"]).difference(set(df_st["TagName"]))
print(len(list(tag_diff)))

In [46]:
ubuntu_dict = XML_extraction(data_per_tag_limit, path, "askubuntu", list(tag_diff))

In [47]:
len(ubuntu_dict.keys())

1519

In [48]:
with open("../StackExchange/other_sites/au_tags_tf.json", "w") as outfile:
    json.dump(ubuntu_dict, outfile)

In [50]:
data_per_tag_limit = 100
path = "../../../../Stackoverflow_dump/serverfault.com/"

df_st = pd.read_csv("../StackExchange/site_tags/stackoveflow_tag.csv")
df_sf = pd.read_csv("../StackExchange/site_tags/serverfault_tag.csv")

tag_diff = set(df_sf["TagName"]).difference(set(df_st["TagName"]))
print(len(list(tag_diff)))

1543


In [51]:
server_dict = XML_extraction(data_per_tag_limit, path, "serverfault", list(tag_diff))

In [53]:
with open("../StackExchange/other_sites/sf_tags_tf.json", "w") as outfile:
    json.dump(server_dict, outfile)

In [55]:
data_per_tag_limit = 100
path = "../../../../Stackoverflow_dump/superuser.com/"

df_st = pd.read_csv("../StackExchange/site_tags/stackoveflow_tag.csv")
df_su = pd.read_csv("../StackExchange/site_tags/superuser_tag.csv")

tag_diff = set(df_su["TagName"]).difference(set(df_st["TagName"]))
print(len(list(tag_diff)))

2713


In [56]:
superuser_dict = XML_extraction(data_per_tag_limit, path, "superuser", list(tag_diff))

In [57]:
with open("../StackExchange/other_sites/su_tags_tf.json", "w") as outfile:
    json.dump(superuser_dict, outfile)

In [59]:
data_per_tag_limit = 100
path = "../../../../Stackoverflow_dump/softwareengineering.stackexchange.com/"

df_st = pd.read_csv("../StackExchange/site_tags/stackoveflow_tag.csv")
df_se = pd.read_csv("../StackExchange/site_tags/SE_tag.csv")

tag_diff = set(df_se["TagName"]).difference(set(df_st["TagName"]))
print(len(list(tag_diff)))

542


In [60]:
se_dict = XML_extraction(data_per_tag_limit, path, "SE", list(tag_diff))

In [61]:
with open("../StackExchange/other_sites/se_tags_tf.json", "w") as outfile:
    json.dump(se_dict, outfile)

## Data selection from json files

In [25]:
import operator

with open(exchange_folder+"Stackoverflow/st_tags_tf.json") as json_file:
    tag_dict = json.load(json_file)
    
sorted_d = dict(sorted(tag_dict.items(), key=operator.itemgetter(1),reverse=True))

In [44]:
sorted_d

{'javascript': 2453713,
 'python': 2071327,
 'java': 1878116,
 'c#': 1571198,
 'php': 1451348,
 'android': 1393126,
 'html': 1156016,
 'jquery': 1031378,
 'c++': 783567,
 'css': 779102,
 'ios': 677311,
 'mysql': 655003,
 'sql': 650208,
 'r': 473420,
 'node.js': 450192,
 'reactjs': 430697,
 'arrays': 403843,
 'c': 390087,
 'asp.net': 371245,
 'json': 350220,
 'ruby-on-rails': 335314,
 'python-3.x': 329120,
 '.net': 326204,
 'sql-server': 325419,
 'swift': 321938,
 'django': 301217,
 'objective-c': 292624,
 'angular': 287960,
 'excel': 272956,
 'pandas': 267949,
 'angularjs': 262603,
 'regex': 254924,
 'ruby': 226594,
 'linux': 221573,
 'iphone': 221564,
 'ajax': 221137,
 'xml': 211315,
 'vba': 205496,
 'spring': 202346,
 'typescript': 202245,
 'laravel': 200885,
 'asp.net-mvc': 199647,
 'database': 190767,
 'wordpress': 188038,
 'string': 179991,
 'mongodb': 169313,
 'wpf': 167268,
 'postgresql': 163216,
 'windows': 161499,
 'xcode': 157326,
 'bash': 149907,
 'flutter': 148030,
 'oracle

In [45]:
tag_selected = {k:v for k,v in sorted_d.items() if v >=50 }

In [46]:
len(tag_selected)

29500

In [68]:
st_folder = "../StackExchange/Stackoverflow/"
path = st_folder+'DataSelection/'
file_list = os.listdir(path)
file_list.reverse()
data_per_tag_limit = 100
tag_dict = {}

for filename in file_list:
    df_new = pd.read_json(path+filename)
    df_new.Body = df_new.Body.fillna(-1)
    df_new.Title = df_new.Title.fillna(-1)
    df_new = df_new[df_new.Body != -1]
    df_new = df_new[df_new.Title != -1]
    df_new = df_new[df_new.PostTypeId == 1]
    df_new = df_new[df_new["AcceptedAnswerId"] != -1]
    df_new = df_new[df_new["AnswerCount"] > 0]
    df_new = df_new[df_new["Score"] >= 0]
    df_new = df_new.reset_index(drop=True)
    
    keep_or_not = []
    
    #print(filename,len(df_new))
    for tag_string in df_new["Tags"]:
        tag_string = tag_string.replace("<","")
        tag_string = tag_string.replace(">"," ")
        overlimit = True
        
        for tag in tag_string.strip().split(" "):
            if tag in tag_selected.keys():
                tag_dict[tag] = tag_dict.setdefault(tag, 0) + 1
                
                if tag_dict[tag] < data_per_tag_limit:
                    overlimit = False
            
        if overlimit:
            keep_or_not.append("N")
        else:
            keep_or_not.append("Y")
    
    df_new["Keep"] = keep_or_not
    length = len(df_new[df_new["Keep"] == "Y"])
    df_new[df_new["Keep"] == "Y"].to_csv(st_folder+"filtered_data/"+filename[:-5]+"_"+str(length)+".csv", index = None)
    

In [69]:
len(tag_dict.keys())

29500

In [83]:
sorted_d = dict(sorted(tag_dict.items(), key=operator.itemgetter(1),reverse=True))

In [84]:
sorted_d

{'java': 540903,
 'python': 479865,
 'javascript': 463134,
 'c#': 435962,
 'android': 420151,
 'php': 295528,
 'c++': 264241,
 'ios': 217560,
 'jquery': 185883,
 'html': 157781,
 'r': 140294,
 'css': 118833,
 '.net': 118496,
 'c': 118431,
 'sql': 114820,
 'ruby-on-rails': 110177,
 'node.js': 105164,
 'mysql': 104883,
 'objective-c': 104026,
 'asp.net': 99795,
 'iphone': 95843,
 'reactjs': 85042,
 'swift': 82113,
 'django': 72986,
 'linux': 72941,
 'ruby': 72501,
 'angular': 68223,
 'json': 64958,
 'sql-server': 64618,
 'angularjs': 62572,
 'python-3.x': 59339,
 'arrays': 58693,
 'xml': 56504,
 'azure': 55569,
 'windows': 54360,
 'wpf': 53455,
 'spring': 52239,
 'amazon-web-services': 50953,
 'git': 48803,
 'asp.net-mvc': 48187,
 'xcode': 47718,
 'macos': 44432,
 'database': 44243,
 'excel': 42714,
 'scala': 42309,
 'postgresql': 41519,
 'regex': 41483,
 'typescript': 41359,
 'eclipse': 41260,
 'multithreading': 40654,
 'laravel': 40442,
 'algorithm': 39242,
 'oracle': 38784,
 'wordpres

In [35]:
path = st_folder+'filtered_data/'
file_list = os.listdir(path)

In [36]:
df_list = [pd.read_csv(path+filename) for filename in file_list]

In [37]:
len(df_list)

861

In [38]:
df_final = pd.concat(df_list)

In [39]:
len(df_final)

2053988

In [40]:
df_final.to_csv("../StackExchange/final_data/stackoverflow_"+str(len(df_final))+".csv", index = None)

### other sites

In [31]:
def data_selection(file):
    df_new = pd.read_json("../StackExchange/other_sites/"+file)
    df_new.Body = df_new.Body.fillna(-1)
    df_new.Title = df_new.Title.fillna(-1)
    df_new = df_new[df_new.Body != -1]
    df_new = df_new[df_new.Title != -1]
    df_new = df_new[df_new.PostTypeId == 1]
    df_new = df_new[df_new["AcceptedAnswerId"] != -1]
    df_new = df_new[df_new["AnswerCount"] > 0]
    df_new = df_new[df_new["Score"] >= 0]
    df_new = df_new.reset_index(drop=True)
    
    
    df_new.to_csv("../StackExchange/final_data/"+file[:-5]+"_"+str(len(df_new))+".csv", index = None)

In [33]:
data_selection("askdifferent.json")

In [49]:
data_selection("askubuntu.json")

In [54]:
data_selection("serverfault.json")

In [58]:
data_selection("superuser.json")

In [62]:
data_selection("SE.json")

## Combine rawdata into one spreadsheet

In [ ]:
df = pd.DataFrame()
path = "../StackExchange/final_data/"

for filename in os.listdir(path):
    df_new = pd.read_csv(path+filename)
    print(filename,len(df_new))
    df = df.append(df_new)

df.to_csv("../StackExchange/final_data/stackexchange.csv",index=None)

In [ ]:
len(df)

In [ ]:
df.head()

### Data Filtering (Optional)

In [3]:
df = pd.read_csv("../StackExchange/final_data/stackexchange.csv")
df_tags = pd.read_csv("../StackExchange/final_data/tag_dict.csv")
df_tags.Corrected = df_tags.Corrected.fillna(-1)

C:\Users\wangs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
tag_corrected = {}

for i in range(len(df_tags)):
    tag_corrected[df_tags.iloc[i]["Tag"]] = df_tags.iloc[i]["Corrected"]

print(len(tag_corrected))

28142


In [5]:
tag_dict ={}

for i, tag_string in enumerate(df["Tags"]):
    tag_string = tag_string.replace("<","")
    tag_string = tag_string.replace(">"," ")
    
    for tag in tag_string.strip().split(" "):
        tag_dict[tag] = tag_dict.setdefault(tag, 0) + 1
        
print(len(tag_dict.keys()))

59779


In [7]:
sorted_d = dict(sorted(tag_dict.items(), key=operator.itemgetter(1),reverse=False))

In [8]:
tag50_selected = {k:v for k,v in sorted_d.items() if v >= 50 }
print(len(tag50_selected))

28143


In [10]:
data_selectedId = []
tag_dict = {}
data_per_tag_limit = 30
tags_list = []

for i, tag_string in enumerate(df["Tags"]):
    tag_string = tag_string.replace("<","")
    tag_string = tag_string.replace(">"," ")
    overlimit = True
    
    for tag in tag_string.strip().split(" "):
        if tag in tag50_selected.keys() and tag in tag_corrected.keys() and tag_corrected[tag] != "no":
            if tag_corrected[tag] != -1:
                tag = tag_corrected[tag]
                
            tag_dict[tag] = tag_dict.setdefault(tag, 0) + 1
            
            if tag_dict[tag] <= data_per_tag_limit:
                overlimit = False
            
            
    if not overlimit:
        data_selectedId.append(i)
        tags_list.append(tag_string.strip().split(" "))

print("select",len(data_selectedId),"samples")
print("contain",len(tag_dict.keys()),"tags")

select 588644 samples
contain 26067 tags


In [11]:
with open("../StackExchange/final_data/selected_id.txt", 'w') as f:
    for tid in data_selectedId:
        f.write("%d\n"%(tid))

In [ ]:
#save tag data
import csv

with open("../StackExchange/final_data/tag_dict.csv", 'w') as f:
    for key in tag_dict.keys():
        f.write("%s,%s\n"%(key,tag_dict[key]))

In [ ]:
df_selected = df.iloc[data_selectedId]
df_selected.to_csv("../StackExchange/final_data/stackexchange_filtered.csv")

## Tag Comparison among different sites

In [ ]:
df_st = pd.read_csv(st_folder+"site_tags/stackoveflow_tag.csv")
df_ad = pd.read_csv(st_folder+"site_tags/askdifferent_tag.csv")
df_au = pd.read_csv(st_folder+"site_tags/askubuntu_tag.csv")
df_se = pd.read_csv(st_folder+"site_tags/SE_tag.csv")
df_sf = pd.read_csv(st_folder+"site_tags/serverfault_tag.csv")
df_su = pd.read_csv(st_folder+"site_tags/superuser_tag.csv")

In [ ]:
print("Stack Oveflow",len(df_st["TagName"]))
print("Ask Different",len(df_ad["TagName"]))
print("Ask Ubuntu",len(df_au["TagName"]))
print("Software Engineering",len(df_se["TagName"]))
print("Server Fault",len(df_sf["TagName"]))
print("Super User",len(df_su["TagName"]))

In [ ]:
print("Stack Oveflow & Ask Different",len(set(df_st["TagName"]).intersection(set(df_ad["TagName"]))))
print("Ask Different has",len(set(df_ad["TagName"]).difference(set(df_st["TagName"]))),"unique tags")
print("Stack Oveflow & Ask Ubuntu",len(set(df_st["TagName"]).intersection(set(df_au["TagName"]))))
print("Ask Ubuntu has",len(set(df_au["TagName"]).difference(set(df_st["TagName"]))),"unique tags")
print("Stack Oveflow & Software Engineering",len(set(df_st["TagName"]).intersection(set(df_se["TagName"]))))
print("Software Engineering has",len(set(df_se["TagName"]).difference(set(df_st["TagName"]))),"unique tags")
print("Stack Oveflow & Server Fault",len(set(df_st["TagName"]).intersection(set(df_sf["TagName"]))))
print("Server Fault has",len(set(df_sf["TagName"]).difference(set(df_st["TagName"]))),"unique tags")
print("Stack Oveflow & Super User",len(set(df_st["TagName"]).intersection(set(df_su["TagName"]))))
print("Super User has",len(set(df_su["TagName"]).difference(set(df_st["TagName"]))),"unique tags")